In [1]:
import re
import mysql.connector
import numpy as np

In [2]:
mydb = mysql.connector.connect(
    host='127.0.0.1', 
    user='Yuxin Xiao', 
    password='19961002XYX',
    database='CS510Project')
mycursor = mydb.cursor()

In [118]:
mycursor.execute("CREATE TABLE NEW_Sentences (Sentence_id INT, Sentence TEXT)")
mycursor.execute("CREATE TABLE NEW_Words (Word TEXT, Count INT, Inverted_index LONGTEXT, Q_inverted_index LONGTEXT)")

In [117]:
mycursor.execute("DROP TABLE NEW_Sentences")
mycursor.execute("DROP TABLE NEW_Words")

In [87]:
def insert_sentences(Id, Sentence):
    sql = ('INSERT INTO NEW_Sentences '
           '(Sentence_id, Sentence) '
           'VALUES (%s, %s)')
    val = (Id, Sentence)
    mycursor.execute(sql, val)

In [88]:
with open("sentences-remove-zeros.txt", "r", encoding='utf-8', errors='ignore') as file:
    sentences = file.read().split('\n')
    for Id, Sentence in enumerate(sentences):
        insert_sentences(Id, Sentence)
        
        if Id % 5000 == 0:
            print(Id)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [89]:
mycursor.execute("SELECT * FROM NEW_Sentences")
myresult = mycursor.fetchall()

In [90]:
print("Creating Dictionary")
print(len(myresult))
   
count_dict = {}
index_dict = {}

num = 0
for row in myresult:
    
    num += 1
    if num % 5000 == 0:
        print(num)
    
    index = str(row[0])
    sentence = re.sub(r'[^\w\s]', "", row[1]).split()
        
    for word in sentence:
        word = word.lower()
        
        if word in count_dict:
            count_dict[word] += 1
            index_dict[word] += (' ' + index)
        else:
            count_dict[word] = 1
            index_dict[word] = index

Creating Dictionary
58148
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [116]:
with open("sentences-remove-zeros-tab-split.txt", "r", encoding='utf-8', errors='ignore') as file:
    myresult = file.read().split('\n')

print("Re-Creating Dictionary")
print(len(myresult))
   
count_dict = {}
index_dict = {}

num = 0
for index, sentence in enumerate(myresult):
    
    num += 1
    if num % 5000 == 0:
        print(num)
    
    sentence = re.sub(r'[^\w\s]', "", sentence).split('\t')
    sentence = list(filter(None, sentence))
        
    for word in sentence:
        word = word.lower()
        
        if word in count_dict:
            count_dict[word] += 1
            index_dict[word] += (' ' + str(index))
        else:
            count_dict[word] = 1
            index_dict[word] = str(index)

Re-Creating Dictionary
58148
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [119]:
q_index_dict = {}

with open("questionables-remove-zeros.txt", "r", encoding='utf-8', errors='ignore') as file:
    sentences = file.read().split('\n')
    for index, sentence in enumerate(sentences):
        if index % 5000 == 0:
            print(index)
        words = re.sub(r'[^\w\s]', "", sentence).split('\t')
        words = list(filter(None, words))
        for word in words:
            word = word.lower()
            if word in q_index_dict:
                q_index_dict[word] += (' ' + str(index))
            else:
                q_index_dict[word] = str(index)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [120]:
print("Writing into Database")
print(len(count_dict.keys()))
print(len(q_index_dict.keys()))

num = 0
for word in count_dict.keys():
    
    num += 1
    if num % 5000 == 0:
        print(num)
    
    count = count_dict[word]
    index = index_dict[word]
    
    if word in q_index_dict:
        q_index = q_index_dict[word]
    else:
        q_index = ""

    insert_sql = "INSERT INTO NEW_Words (Word, Count, Inverted_index, Q_inverted_index) VALUES (%s, %s, %s, %s)"
    insert_val = (word, count, index, q_index)
    mycursor.execute(insert_sql, insert_val)

Writing into Database
70033
39351
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000


In [121]:
select_sql = "SELECT * FROM NEW_Words WHERE Word = %s"
select_val = ("mother",)
mycursor.execute(select_sql, select_val)
myresult = mycursor.fetchall()

In [122]:
len(myresult)

1

In [123]:
myresult[0]

('mother',
 202,
 '14 17 40 50 88 95 205 253 255 260 275 303 305 364 379 381 487 1199 1755 1777 1777 1779 1821 2066 2068 2069 2072 2120 2196 2203 2206 2248 2258 2259 2959 5590 5603 5605 5606 5607 5636 6248 7044 7170 7379 8629 8878 9515 9524 9978 10038 12247 12251 12253 12373 12376 13068 15904 16717 16727 16741 16742 16742 16743 16746 16755 16755 16766 16774 16778 16778 16781 16786 16789 16794 16807 16808 17025 17208 19085 19095 19115 19123 19180 19180 19183 19368 20178 20614 20661 20669 20688 21224 21225 21226 21233 21234 21234 21234 21253 21307 21308 21337 21341 21355 21355 21356 21357 21379 21384 21393 21873 21877 22603 23217 24072 24105 24246 25802 25837 26776 26778 26781 26782 26783 26987 26988 26989 26989 26990 27343 27404 27431 29027 29127 29188 30472 30494 32621 33321 33335 35817 37108 37419 37446 37825 37959 38670 39916 39917 40527 40571 40594 40605 42591 42598 42950 43185 43291 43423 44032 44521 44524 44526 44635 44957 45089 45225 46610 46643 46754 47944 48061 50902 50903 5135

In [124]:
mydb.commit()

In [125]:
mycursor.close()
mydb.close()